[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Strzabala_2025_BEng/paraview.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Strzabala_2025_BEng/paraview.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Strzabala_2025_BEng/paraview.ipynb)

Runs a parcel simulation based on Pyrcel documentation setup, and demonstrates export to VTK and visualisation using Paraview

(requires `pvpython` command in PATH)

In [1]:
import os, sys
os.environ['NUMBA_THREADING_LAYER'] = 'workqueue'  # PySDM & PyMPDATA don't work with TBB; OpenMP has extra dependencies on macOS
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples', 'PySDM')

In [7]:
import numpy as np
import subprocess
import platform
import pathlib
from PySDM import Formulae
from PySDM.physics import si
from PySDM.initialisation.spectra import Lognormal
from PySDM.products import (
    ParcelDisplacement,
    AmbientTemperature,
    AmbientDryAirDensity,
    AmbientRelativeHumidity,
    ParticleSizeSpectrumPerVolume,
    ParticleVolumeVersusRadiusLogarithmSpectrum,
)
from PySDM.exporters import VTKExporterParcel

from PySDM_examples.Pyrcel import Settings, Simulation
import PySDM_examples

In [3]:
settings = Settings(
    dz=10 * si.m,
    n_sd_per_mode=(25, 25),
    aerosol_modes_by_kappa={
        0.54: Lognormal(
            norm_factor=850 / si.cm**3,
            m_mode=15 * si.nm,
            s_geom=1.6,
        ),
        1.2: Lognormal(
            norm_factor=10 / si.cm**3,
            m_mode=850 * si.nm,
            s_geom=1.2,
        ),
    },
    vertical_velocity=1.0 * si.m / si.s,
    initial_pressure=775 * si.mbar,
    initial_temperature=274 * si.K,
    initial_relative_humidity=0.90,
    displacement=1000 * si.m,
    formulae=Formulae(constants={"MAC": 0.3}),
)

dry_radius_bin_edges = np.logspace(
    np.log10(1e-3 * si.um), np.log10(5e0 * si.um), 33, endpoint=False
)

simulation = Simulation(
    settings,
    products=(
        ParcelDisplacement(name="z"),
        AmbientRelativeHumidity(name="S_max_percent", unit="%", var="RH"),
        AmbientTemperature(name="T"),
        ParticleSizeSpectrumPerVolume(
            name="dry:dN/dR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        ParticleVolumeVersusRadiusLogarithmSpectrum(
            name="dry:dV/dlnR", radius_bins_edges=dry_radius_bin_edges, dry=True
        ),
        AmbientDryAirDensity(),
    ),
    mass_of_dry_air = 66666 * si.kg,
)

In [4]:
output = simulation.run()

In [5]:
e = VTKExporterParcel(n_sd=simulation.particulator.n_sd, output=output, mass_of_dry_air=simulation.particulator.environment.mass_of_dry_air)
for step in settings.output_steps:
    e.export_products(step, simulation)
    e.export_attributes(step, simulation)
e.write_pvd()

In [ ]:
product = pathlib.Path("./output/sd_products.pvd").absolute()
attributes = pathlib.Path("./output/sd_attributes.pvd").absolute()

paraview_script = pathlib.Path(PySDM_examples.__file__).parent / 'Strzabala_2025_BEng' / "paraview_parcel_model.py"

args = [
    "pvpython",
    "--force-offscreen-rendering",
    str(paraview_script),
    "--sd-products-pvd",
    str(product),
    "--sd-attributes-pvd",
    str(attributes),
    "--output-animation-path",
    str(pathlib.Path("./output/parcel_animation.ogv").absolute()),
    "--output-screenshot-path",
    str(pathlib.Path("./output/last_frame.pdf").absolute()),
]
result = subprocess.run(
    args,
    check=platform.system() != "Windows",
    capture_output=True,
    text=True,
)